In [2]:
import os
import sys
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.io import loadmat
import warnings
warnings.simplefilter("ignore")

# Import Pyod and the methods

In [3]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

# Import Metrics Package

In [4]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores
from sklearn.metrics import roc_auc_score

In [5]:
mat_file_list=['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satellite.mat','satimage-2.mat','shuttle.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [6]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satellite.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

# Loading mat file

In [7]:
df_columns=['Data','#Sample','#Dimensions','Outlier Perc','PCA','MCD','OCSVM','LOF','CBLOF','KNN','HBOS','ABOD','IFOREST','FEATUREBAGGING']

# Precision Time and Roc evolution tables creation

In [8]:
roc_df=pd.DataFrame(columns=df_columns)
prn_df=pd.DataFrame(columns=df_columns)
time_df=pd.DataFrame(columns=df_columns)
print(roc_df,prn_df,time_df)

Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: [] Empty DataFrame
Columns: [Data, #Sample, #Dimensions, Outlier Perc, PCA, MCD, OCSVM, LOF, CBLOF, KNN, HBOS, ABOD, IFOREST, FEATUREBAGGING]
Index: []


# Exploring Mat files

In [10]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n... Processing",mat_file,'.....')
    mat = loadmat(mat_file)
    
    X = mat['X']
    y = mat['y'].ravel()
    
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100, ndigits=4)
    
    roc_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    prn_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    time_list = [mat_file[:-4], X.shape[0], X.shape[1], outliers_percentage]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state = random_state)
    
    X_train_norm, X_test_norm = standardizer(X_train, X_test)
    
    classifiers = {'Angle-based Outlier Detector (ABOD)' : ABOD(contamination=outliers_fraction), 
                  'Cluster-based Local Outlier Factor': CBLOF(contamination=outliers_fraction,check_estimator=False ,random_state=random_state),
                  'Feature Bagging': FeatureBagging(contamination=outliers_fraction, random_state=random_state),
                  'Historgram-base Outlier Detection (HBOS)': HBOS(contamination=outliers_fraction),
                  'Isolation Forest' : IForest(contamination=outliers_fraction, random_state=random_state),
                  'K Nearest Neighbors (KNN)': KNN(contamination=outliers_fraction),
                  'Local Outlier Factor (LOF)' : LOF(contamination=outliers_fraction),
                  'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction, random_state=random_state),
                  'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
                  'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction, random_state=random_state)}
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0, 4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test, test_scores), 4)
        prn = round(precision_n_scores(y_test, test_scores), 4)
        
        print(f"{clf_name} ROC: {roc}, precision @ rank n: {prn}, execution time: {duration}s' ")
        
        roc_list.append(roc)
        prn_list.append(prn)
        
    temp_df = pd.DataFrame(time_list).transpose()
    temp_df.columns = df_columns
    time_df = pd.concat([time_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(roc_list).transpose()
    temp_df.columns = df_columns
    roc_df = pd.concat([roc_df, temp_df], axis=0)
    
    temp_df = pd.DataFrame(prn_list).transpose()
    temp_df.columns = df_columns
    prn_df = pd.concat([prn_df, temp_df], axis=0)


... Processing arrhythmia.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7687, precision @ rank n: 0.3571, execution time: 0.136s' 
Cluster-based Local Outlier Factor ROC: 0.7684, precision @ rank n: 0.4643, execution time: 0.096s' 
Feature Bagging ROC: 0.7799, precision @ rank n: 0.5, execution time: 0.576s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.8511, precision @ rank n: 0.5714, execution time: 0.056s' 
Isolation Forest ROC: 0.8478, precision @ rank n: 0.5357, execution time: 0.392s' 
K Nearest Neighbors (KNN) ROC: 0.782, precision @ rank n: 0.5, execution time: 0.0841s' 
Local Outlier Factor (LOF) ROC: 0.7787, precision @ rank n: 0.4643, execution time: 0.072s' 
Minimum Covariance Determinant (MCD) ROC: 0.8228, precision @ rank n: 0.4286, execution time: 2.2839s' 
One-class SVM (OCSVM) ROC: 0.7986, precision @ rank n: 0.5, execution time: 0.052s' 
Principal Component Analysis (PCA) ROC: 0.7997, precision @ rank n: 0.5, execution time: 0.108s' 

... Processing cardio

K Nearest Neighbors (KNN) ROC: 0.3824, precision @ rank n: 0.0, execution time: 1.572s' 
Local Outlier Factor (LOF) ROC: 0.4584, precision @ rank n: 0.0484, execution time: 1.452s' 
Minimum Covariance Determinant (MCD) ROC: 0.3486, precision @ rank n: 0.0, execution time: 3.716s' 
One-class SVM (OCSVM) ROC: 0.4972, precision @ rank n: 0.0, execution time: 1.416s' 
Principal Component Analysis (PCA) ROC: 0.504, precision @ rank n: 0.0, execution time: 0.088s' 

... Processing pendigits.mat .....
Angle-based Outlier Detector (ABOD) ROC: 0.7008, precision @ rank n: 0.0308, execution time: 1.448s' 
Cluster-based Local Outlier Factor ROC: 0.9609, precision @ rank n: 0.3077, execution time: 0.212s' 
Feature Bagging ROC: 0.4687, precision @ rank n: 0.0462, execution time: 4.184s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.9294, precision @ rank n: 0.2615, execution time: 0.008s' 
Isolation Forest ROC: 0.9482, precision @ rank n: 0.2615, execution time: 0.624s' 
K Nearest Neighbors (KNN)

Angle-based Outlier Detector (ABOD) ROC: 0.9573, precision @ rank n: 0.5556, execution time: 0.1462s' 
Cluster-based Local Outlier Factor ROC: 0.9751, precision @ rank n: 0.6667, execution time: 0.1542s' 
Feature Bagging ROC: 0.979, precision @ rank n: 0.7778, execution time: 0.1631s' 
Historgram-base Outlier Detection (HBOS) ROC: 0.9876, precision @ rank n: 0.6667, execution time: 0.02s' 
Isolation Forest ROC: 0.9674, precision @ rank n: 0.5556, execution time: 0.7293s' 
K Nearest Neighbors (KNN) ROC: 0.9713, precision @ rank n: 0.4444, execution time: 0.0439s' 
Local Outlier Factor (LOF) ROC: 0.979, precision @ rank n: 0.6667, execution time: 0.0161s' 
Minimum Covariance Determinant (MCD) ROC: 0.9658, precision @ rank n: 0.5556, execution time: 0.4252s' 
One-class SVM (OCSVM) ROC: 0.9782, precision @ rank n: 0.6667, execution time: 0.012s' 
Principal Component Analysis (PCA) ROC: 0.9775, precision @ rank n: 0.6667, execution time: 0.004s' 


In [11]:
from IPython.core.display import HTML
HTML("""
<style>
    table.dataframe thead th:first-child {
        display: none;
    }
    table.dataframe tbody th {
        display: none;
    }
</style>
""")

In [12]:
roc_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,arrhythmia,452,274,14.6018,0.7687,0.7684,0.7799,0.8511,0.8478,0.782,0.7787,0.8228,0.7986,0.7997
0,cardio,1831,21,9.6122,0.5763,0.8221,0.4879,0.8453,0.9316,0.6959,0.4715,0.8778,0.9507,0.9638
0,glass,214,9,4.2056,0.7104,0.8506,0.7043,0.6524,0.7195,0.7805,0.7774,0.7165,0.6189,0.622
0,ionosphere,351,33,35.8974,0.9004,0.8952,0.8933,0.5195,0.8294,0.9134,0.8989,0.9399,0.8372,0.7971
0,letter,1600,32,6.25,0.8465,0.7423,0.866,0.5728,0.5836,0.845,0.8409,0.7499,0.5744,0.48
0,lympho,148,18,4.0541,0.9382,0.9709,0.9673,0.9964,0.9855,0.9636,0.9636,0.9164,0.9636,0.9818
0,mnist,7603,100,9.2069,0.7813,0.8447,0.7259,0.5675,0.7813,0.8409,0.7085,0.863,0.8417,0.8396
0,musk,3062,166,3.1679,0.0809,1,0.5228,0.9999,0.9992,0.7348,0.5323,1,1,1
0,optdigits,5216,64,2.8758,0.4428,0.7852,0.4641,0.8822,0.5442,0.3824,0.4584,0.3486,0.4972,0.504


In [13]:
prn_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,arrhythmia,452,274,14.6018,0.3571,0.4643,0.5,0.5714,0.5357,0.5,0.4643,0.4286,0.5,0.5
0,cardio,1831,21,9.6122,0.1875,0.4844,0.1406,0.4688,0.4531,0.2812,0.125,0.3906,0.5938,0.6875
0,glass,214,9,4.2056,0.25,0.25,0.25,0,0.25,0.25,0.25,0,0.25,0.25
0,ionosphere,351,33,35.8974,0.8214,0.8036,0.75,0.3393,0.6607,0.8393,0.75,0.8571,0.7143,0.5893
0,letter,1600,32,6.25,0.275,0.175,0.4,0.125,0.05,0.3,0.325,0.075,0.1,0.05
0,lympho,148,18,4.0541,0.4,0.6,0.6,0.8,0.6,0.6,0.6,0.6,0.6,0.8
0,mnist,7603,100,9.2069,0.3562,0.4007,0.3664,0.1199,0.3116,0.4144,0.339,0.3973,0.3801,0.3767
0,musk,3062,166,3.1679,0.0333,1,0.1667,0.9667,0.9,0.2333,0.1333,0.9667,1,1
0,optdigits,5216,64,2.8758,0.0161,0,0.0484,0.2581,0.0161,0,0.0484,0,0,0


In [14]:
time_df

,Data,#Sample,#Dimensions,Outlier Perc,PCA,MCD,OCSVM,LOF,CBLOF,KNN,HBOS,ABOD,IFOREST,FEATUREBAGGING
0,arrhythmia,452,274,14.6018,2.8259,4.9304,0.5601,2.7263,1.0689,0.084,0.072,5.6675,0.064,0.132
0,arrhythmia,452,274,14.6018,0.136,0.096,0.576,0.056,0.392,0.0841,0.072,2.2839,0.052,0.108
0,cardio,1831,21,9.6122,0.316,0.288,0.724,0.004,0.376,0.14,0.088,1.9294,0.064,0.004
0,glass,214,9,4.2056,0.032,0.044,0.0401,0.004,0.288,0.008,0,0.8718,0,0
0,ionosphere,351,33,35.8974,0.052,0.04,0.064,0.008,0.308,0.016,0.008,0.168,0.008,0.004
0,letter,1600,32,6.25,0.352,0.084,0.676,0.008,0.3841,0.124,0.08,2.528,0.076,0.012
0,lympho,148,18,4.0541,0.028,0.036,0.028,0.0079,0.288,0.004,0,0.0557,0,0
0,mnist,7603,100,9.2069,6.3,0.54,48.168,0.044,1.876,6.912,6.16,9.0319,4.18,0.156
0,musk,3062,166,3.1679,1.924,0.22,11.988,0.052,1.168,1.564,1.492,46.1519,1.116,0.196
0,optdigits,5216,64,2.8758,2.148,0.324,12.72,0.028,0.864,1.572,1.452,3.716,1.416,0.088
